# Input data Preparation

TODO: Explain the math

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
close_df = pd.read_csv('..\data\DK\preprocessed_data\danish_closed_stocks.csv', index_col=0)
close_df = close_df.fillna(method='bfill')

In [3]:
close_df.index = pd.to_datetime(close_df.index)

In [4]:
df_weekly_prices = close_df.resample('W').last()

In [5]:
df_weekly_log_returns = np.log(df_weekly_prices) - np.log(df_weekly_prices.shift(1))
df_weekly_log_returns = df_weekly_log_returns.dropna(how='all')
df_weekly_log_returns = df_weekly_log_returns.fillna(0)

In [6]:
return_array = df_weekly_log_returns.to_numpy()
m_weekly_log = np.mean(return_array, axis=0)
S_weekly_log = np.cov(return_array.transpose())

In [7]:
m_log = 52 * m_weekly_log
S_log = 52 * S_weekly_log

In [8]:
p_0 = df_weekly_prices.iloc[0].to_numpy()
m_P = p_0 * np.exp(m_log + 1/2*np.diag(S_log))
S_P = np.outer(m_P, m_P) * (np.exp(S_log) - 1)

In [10]:
m = 1 / p_0 * m_P - 1
S = 1 / np.outer(p_0, p_0) * S_P

In [11]:
# Save m and S to some files 
np.save('data/m_log.npy', m_log)
np.save('data/S_log.npy', S_log)
np.save('data/m.npy', m)
np.save('data/S.npy', S)